In [1]:
import os
from dotenv import load_dotenv

import pandas as pd

from h2ogpte import H2OGPTE

In [2]:
#!cat .env

# CBM_default
# CBM_structured
# CBM_structured_lChunk

In [3]:
load_dotenv(dotenv_path='.env')
heogpteKey_CBM_default = os.getenv('CBM_default')
if heogpteKey_CBM_default: print('key is set.')

key is set.


In [4]:
client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpteKey_CBM_default,
)
client

In [5]:
# 利用可能なLLM
client.get_llm_names()

['meta-llama/Meta-Llama-3.1-8B-Instruct',
 'h2oai/h2o-danube3-4b-chat',
 'Qwen/Qwen2-VL-7B-Instruct',
 'meta-llama/Meta-Llama-3.1-70B-Instruct',
 'meta-llama/Meta-Llama-3.1-405B-Instruct-FP8',
 'Qwen/Qwen2.5-72B-Instruct',
 'Qwen/Qwen2-VL-72B-Instruct',
 'h2oai/h2ovl-mississippi-2b',
 'mistralai/Pixtral-12B-2409',
 'mistralai/Mixtral-8x7B-Instruct-v0.1',
 'meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo',
 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
 'upstage/SOLAR-10.7B-Instruct-v1.0',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo',
 'meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo',
 'meta-llama/Llama-3.2-3B-Instruct-Turbo',
 'claude-3-haiku-20240307',
 'claude-3-5-haiku-20241022',
 'claude-3-5-sonnet-20241022',
 'claude-3-5-sonnet-20241022-bedrock',
 'gpt-4o',
 'gpt-4o-mini',
 'mistral-tiny',
 'mistral-small-latest',
 'mistral-medium-latest',
 'mistral-large-latest',
 'gemini-1.5-pro-latest',
 'gemini-1.5-flash-latest']

In [34]:
#client.get_default_collection()

In [35]:
#client.get_prompt_template()

In [18]:
client.count_prompt_templates()

28

In [33]:
#client.list_recent_prompt_templates(offset=0, limit=100)

In [42]:
client.get_prompt_template('4d3a78f7-798f-44d9-b577-5304bc07f255')

PromptTemplate(is_default=True, id='4d3a78f7-798f-44d9-b577-5304bc07f255', name='デフォルト', description='デフォルトの h2oGPTe プロンプト テンプレート', lang='ja', system_prompt='あなたは H2O.ai によって作成された専門家の質問を消費する AI システムである h2oGPTe です。', pre_prompt_query='以下の情報に注意して覚えておいてください。最後に質問や命令に答えるためには、チャット履歴、指定された画像、または文書テキストのみを使用する必要があります。', prompt_query='チャット履歴の情報のみに従って、与えられた画像、または上記のコンテキスト内で提供されているドキュメントテキストは、次のように明確に構成された応答（「従い」から始まる）を示します。', hyde_no_rag_llm_prompt_extension='回答は簡潔にして、最後に最も関連性の高いキーワードを 5 つ挙げてください。', pre_prompt_summary='簡潔な 1 段落または箇条書きの要約を作成するには、チャット履歴、指定された画像、または次のテキストに注意してください。', prompt_summary='チャット履歴、指定された画像、または上記のテキストのみを使用して、重要な結果の凝\u200b\u200b縮された簡潔な適切に構造化されたマークダウンの概要を記述します。', system_prompt_reflection='あなたは裁判官を務めています。公平かつ公平であり、細部に注意を払わなければなりません。', pre_prompt_reflection='', prompt_reflection='以下で説明する加法的な 5 ポイント スコアリング システムを使用して、ユーザーの質問と対応する応答を確認します。ポイントは、各基準の満足度に基づいて蓄積されます。\n\n- 回答が不完全であったり無関係な内容が含まれていたとしても、回答が関連性があり、ユーザーの問い合わせに関連する情報を提供している場合は 1 ポイントを追加します。\n- 回答がユーザーの質問のかなりの部分に対応しているものの、質問

In [7]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'a72f9a5a-0c25-4ed7-92fa-fdc686d730c0'

In [41]:
#client.get_chat_session_prompt_template('a72f9a5a-0c25-4ed7-92fa-fdc686d730c0')

In [43]:
%%time

#LlmModel = 'Qwen/Qwen2.5-72B-Instruct'
#LlmModel = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
#LlmModel = 'gpt-4o-mini'
LlmModel = 'gpt-4o'

system_prompt = '''
あなたは質問に対し、与えられたドキュメントから回答を実施する優秀なRAGチャットアシスタントです。
必ず日本語で回答します。
長くても70トークン以内で簡潔に回答を作成してください。回答を得た理由を述べる必要はないです。
'''
pre_prompt_query='以下の情報に注意して覚えておいてください。最後に質問や命令に答えるためには、チャット履歴、指定された画像、または文書テキストのみを使用する必要があります。'
prompt_query='チャット履歴の情報のみに従って、与えられた画像、または上記のコンテキスト内で提供されているドキュメントテキストは、次のように明確に構成された応答（「従い」から始まる）を示します。'

message = 'パイントグラスに2段階で注ぎ、泡がわずかに盛り上がったら注ぎが完了するビールは？'

print('>>>>> Prompt')
print(Message)

print('>>>>> LLM Reply')
with client.connect(chat_session_id) as session:
    reply = session.query(
        message = message,
        system_prompt=system_prompt,
        pre_prompt_query=pre_prompt_query,
        prompt_query=prompt_query,
        llm=LlmModel,
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
    )
    print(reply.content)

>>>>> Prompt
パイントグラスに2段階で注ぎ、泡がわずかに盛り上がったら注ぎが完了するビールは？
>>>>> LLM Reply
従い、パイントグラスに2段階で注ぎ、泡がわずかに盛り上がったら注ぎが完了するビールは「NITRO BEER」です。
CPU times: user 19.7 ms, sys: 4.44 ms, total: 24.1 ms
Wall time: 12.1 s


In [44]:
!ls

QandA.xlsx     Untitled.ipynb test.ipynb     test2.ipynb


In [45]:
df = pd.read_excel('QandA.xlsx')
df.shape

(25, 5)

In [46]:
df.head()

,No,Question,Answer,ページ,コメント
0,1,ビールの主な原材料を全て教えてください？,麦芽、ホップ、酵母、水、副原料,4,NaN
1,2,ビールの製造工程の3番目の工程は？,煮沸,5,NaN
2,3,Craft Beer Marketでは何種類のビールを提供していますか？,9種類,"6,7",NaN
3,4,Craft Beer Marketが提供するビールの中で、アルコール度数の最も高いと考えられ...,SPECIAL BEER,7,NaN
4,5,代表的なエールスタイルのビールを３つ教えてください？,「アメリカン・ペールエール」「イングリッシュ・ペールエー\nル」「アンバーエール」「レッドエ...,6,NaN


In [55]:
LlmModel = 'gpt-4o'

res = []

system_prompt = '''
あなたは質問に対し、与えられたドキュメントから回答を実施する優秀なRAGチャットアシスタントです。
必ず日本語で回答します。
長くても70トークン以内で簡潔に回答を作成してください。回答を得た理由を述べる必要はないです。
'''
pre_prompt_query='以下の情報に注意して覚えておいてください。最後に質問や命令に答えるためには、チャット履歴、指定された画像、または文書テキストのみを使用する必要があります。'
prompt_query='チャット履歴の情報のみに従って、与えられた画像、または上記のコンテキスト内で提供されているドキュメントテキストは、次のように明確に構成された応答（「与えられた情報によると、」から始まる）を示します。'

#message = 'パイントグラスに2段階で注ぎ、泡がわずかに盛り上がったら注ぎが完了するビールは？'

for q in df['Question'][:3]:
    print('Question > ', q)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = q,
            system_prompt=system_prompt,
            pre_prompt_query=pre_prompt_query,
            prompt_query=prompt_query,
            llm=LlmModel,
            rag_config={"rag_type": "rag"},   # (Normal) RAG 
        )
        print('Answer > ', reply.content)
        res.append(reply.content)

print(len(res))

Question >  ビールの主な原材料を全て教えてください？
Answer >  与えられた情報によると、ビールの主な原材料は以下の通りです：

1. 麦芽（主に二条大麦）
2. ホップ
3. 酵母
4. 水
5. 副原料（麦、米、糖類、コーンスターチ、こうりやん、ばれいしょ、着色料など）
Question >  ビールの製造工程の3番目の工程は？
Answer >  与えられた情報によると、ビールの製造工程の3番目の工程は「煮沸」です。
Question >  Craft Beer Marketでは何種類のビールを提供していますか？
Answer >  与えられた情報によると、Craft Beer Marketでは以下の種類のビールを提供しています：

1. KANPAI! STANDARD BEER
2. LAGER BEER / PILSNER
3. ALE BEER
4. WHITE BEER / WEIZEN
5. FRUIT/HERB BEER
6. BLACK BEER
7. IPA – INDIA PALE ALE
8. REAL ALE
9. SPECIAL BEER

合計で9種類のビールを提供しています。
3


In [56]:
res

['与えられた情報によると、ビールの主な原材料は以下の通りです：\n\n1. 麦芽（主に二条大麦）\n2. ホップ\n3. 酵母\n4. 水\n5. 副原料（麦、米、糖類、コーンスターチ、こうりやん、ばれいしょ、着色料など）',
 '与えられた情報によると、ビールの製造工程の3番目の工程は「煮沸」です。',
 '与えられた情報によると、Craft Beer Marketでは以下の種類のビールを提供しています：\n\n1. KANPAI! STANDARD BEER\n2. LAGER BEER / PILSNER\n3. ALE BEER\n4. WHITE BEER / WEIZEN\n5. FRUIT/HERB BEER\n6. BLACK BEER\n7. IPA – INDIA PALE ALE\n8. REAL ALE\n9. SPECIAL BEER\n\n合計で9種類のビールを提供しています。']